On souhaite prédire les valeurs de Response avec un modèle Bayésien

L'objectif de ce notebook est de tester l'efficacité d'un modèle Bayésien pour prédire la catégorie à laquelle une personne sera associée en fonction des information qui le caractérisent.
<br>
Ce notebook fait suite a l'analyse exploratoire faite au lien suivant: https://www.kaggle.com/alexdarge/approche-bay-sienne

# Plan
<a id="top"></a>

<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list"  role="tab" aria-controls="home">Sommaire</h3>
    
<font size=+1><b>Feuille de route</b></font>
    
<font size=+1><b>Chargement des données</b></font>
* [Import des fichiers](#0)
* [Import des librairies](#1)

<font size=+1><b>Préparation avant modélisation</b></font>
* [Nettoyage de données](#2)
* [Encodage de la variable catégorielle](#3)
* [Sélection des variables pour modélisation](#4)

    
<font size=+1><b>Bayésien naïf</b></font>
* [Une 1ère prédiction](#5)
* [Modèle sans la variable Product_Info_2](#6)
* [Matrice de confusion sur le 2ème modèle](#7)
 
        
<font size=+1><b>Réduction dimensionnelles</b></font>
* [t-SNE](#8)
* [ACP](#9)

    
<font size=+1><b>Tests</b></font>
* [Tests infructueux](#10)


# Feuille de route
## Fait:
* Bayésien naif sur l'ensemble des variables
* Bayésien sur les variables numériques
* Réduction dimensionnelle avec PCA et t-SNE
* Représentation 

## A faire:
* Identifier une métrique de contrôle afin de comparer avec d'autres modèles prédictifs
* Prédiction avec les features réduites
* Améliorer la performance du modèle

<a id="0"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Sommaire</a>
## Import des fichiers

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<a id="1"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Sommaire</a>
## Import des librairies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from category_encoders.target_encoder import TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn import manifold

import plotly.express as px
from sklearn.decomposition import PCA
from sklearn import decomposition




%matplotlib inline

# Préparation avant modélisation

<a id="2"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Sommaire</a>
## Nettoyage des données

In [ ]:
train = pd.read_csv('../input/prudential-life-insurance-assessment/train.csv.zip')
test = pd.read_csv('../input/prudential-life-insurance-assessment/test.csv.zip')

In [ ]:
# fonction données manquantes
def missing(data):
    total = data.isnull().sum()
    percent = (data.isnull().sum()/data.isnull().count()*100)
    tt = pd.concat([total, percent], axis=1, keys=['Total', 'Pourcent'])
    types = []
    for col in data.columns:
        dtype = str(data[col].dtype)
        types.append(dtype)
    tt['Types'] = types
    return tt
missing(train)['Pourcent'].sort_values(ascending=False)
# suppression des features données manquantes
train_modified = train[train.columns[train.isnull().mean() <= 0.75]]

In [ ]:
# drop données non renseignées
cols_with_missing = [col for col in train_modified.columns 
                                 if train_modified[col].isnull().any()]

In [ ]:
data = train_modified.copy()

<a id="3"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Sommaire</a>
## Encodage de la variable catégorielle

In [ ]:
train.head()

On voit que la variable Product_Info_2 est catégorielle, on s'y intéresse en vue de l'encoder numériquement

In [ ]:
print(len(train.Product_Info_2.unique()))
train.Product_Info_2.unique()

On voit que la variable Product_Info_2 est définie comme une catégorie, elle prend 19 valeurs différentes. Ces valeurs sont l'association d'une lettre et d'un chiffre. Pour l'interpréter numériquement on doit l'encoder c'est à dire créer des features supplémentaires (pour chacune des 19 valeurs). Ces variables seront des booléens qui indiquent quelle valeur de Product_Info_2 la personne a indiqué.
<br>
Par exemple si une personne est caracterisée par la variable Product_Info_2 renseignée comme: 'D3', alors l'encodage des variables encodée sera comme suit: Product_Info_2_D3 vaudra 1 et tous les autres vaudront 0.

In [ ]:
# encodage pour la variable catégorielle
encoded_train = pd.get_dummies(train_modified)
encoded_train.sample(2)

<a id="4"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Sommaire</a>
## Sélection des variables pour la modélisation

In [ ]:
# suppression des colonnes données manquantes
train_modified = encoded_train.drop(cols_with_missing, axis=1)

In [ ]:
# selection de la variable cible pour la modélisation
y=train_modified.Response

In [ ]:
# imputation des données manquantes
my_imputer = SimpleImputer()
imputed_data_train = my_imputer.fit_transform(train_modified)

In [ ]:
# sélection des variables pour la modélisation
df_features = train_modified.loc[:, train_modified.columns != 'Response']

In [ ]:
# formalisation et vérification des dimensions avant modélisation
features=list(df_features.columns)
X=train_modified[features]
print(X.shape)
print(y.shape)

In [ ]:
# suppression de la variable id qui est attribué pour chaque individu 
X=X.drop(['Id'], axis=1)
X

# Bayésien naïf

In [ ]:
# configuation du modèle
Bayes = MultinomialNB()
Bayes.fit(X,y)

<a id="5"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Sommaire</a>
## Un premier résultat

In [ ]:
# affichage comparatif
predict_df=pd.DataFrame(data=Bayes.predict(X))
predict_df.rename(columns={0:'Predicted'}, inplace=True)
predict_df['Response']=y
predict_df

In [ ]:
# nombre des valeurs pour la variable prédite
predict_df.Predicted.value_counts()

In [ ]:
# valeur des prédictions
sns.countplot(data=predict_df, x='Predicted').set_title("Prédiction pour chaque catégorie avec variables encodées")
plt.show()


In [ ]:
# Comparaison prédiction avec les valeurs réelles
fig, axes = plt.subplots(1,2,figsize=(16,6))
fig.suptitle('Comparaison classes prédites vs classes réelles pour le 1er modèle')
sns.countplot(ax=axes[0], data=predict_df, x='Predicted')
sns.countplot(ax=axes[1], data=predict_df, x='Response')
plt.show()

In [ ]:
# différence entre la variable prédite et Response
# intégration de la variable de classe d'écart entre prédiction et valeur réelle
predict_df['diff']=abs(predict_df.Predicted-predict_df.Response)
predict_df.sample(5)

In [ ]:
# différence de classe entre prédiction et Response
predict_df['diff'].value_counts()

In [ ]:
# quantité de données différence classes prédiction/réelles
sns.countplot(data=predict_df, x='diff').set_title("Classes d'écart entre valeurs prédites et valeurs réelles 1er modèle")
plt.show()


In [ ]:
# quantité de classe d'écart
predict_df['diff'].value_counts(normalize=True)

On n'obtient que 37% de résultat exact.
On peut expliquer cela à cause de l'encodage de la variable catégorielle qui créé beaucoup de variables numériques, ce qui peut biaiser la prédiction.
<br>
On constate cependant qu'en cumulé on obtient près de 71% de précision à deux classes d'écart et 54% à une classe d'écart.

On ne prendra pas en considération la variable catégorielle dans le test suivant.

<a id="6"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Sommaire</a>
## Modèle sans la variable Product_Info_2

In [ ]:
data.head()

In [ ]:
data = data.drop('Product_Info_2', axis=1)

In [ ]:
data.head()

In [ ]:
# suppression des données non renseignées
cols_with_missing = [col for col in data.columns 
                                 if data[col].isnull().any()]

In [ ]:
data = data.drop(cols_with_missing, axis=1)
final_df = data.copy()

In [ ]:
y=data.Response

In [ ]:
features=list(data.columns)
X=data[features]
X=X.drop(['Id'], axis=1)
print(X.shape)
print(y.shape)

In [ ]:
Bayes2 = MultinomialNB()
Bayes2.fit(X,y)

In [ ]:
# affichage comparatif
predict2=pd.DataFrame(data=Bayes2.predict(X))
predict2.rename(columns={0:'Predicted'}, inplace=True)
predict2['Response']=y
predict2

In [ ]:
# nombre des valeurs pour la variable prédite
predict_df.Predicted.value_counts() 

In [ ]:
# valeur des prédictions
sns.countplot(data=predict2, x='Predicted').set_title("Prédiction pour chaque catégorie pour le deuxieme modèle")
plt.grid(linestyle='dotted')
plt.show()

In [ ]:
# comparaison du modèle avec la valeurs réelles
fig, axes = plt.subplots(1,2,figsize=(16,6))
fig.suptitle('Comparaison classes prédites vs classes réelles 2ème modèle')
sns.countplot(ax=axes[0], data=predict2, x='Predicted')
sns.countplot(ax=axes[1], data=predict2, x='Response')
plt.show()

In [ ]:
# différence entre la variable prédite et Response
predict2['diff']=abs(predict2.Predicted-predict2.Response)
predict2.sample(5)

In [ ]:
# différence de classe entre prédiction et Response
predict2['diff'].value_counts()

In [ ]:
# quantité de données différence classes prédiction/réelles
sns.countplot(data=predict2, x='diff').set_title("classes d'écart entre valeurs prédites et valeurs réelles 2ème modèle")
plt.grid(linestyle='dotted')
plt.show()

In [ ]:
# pourcentage de classes écart entre prédiction et 'Response'
predict2['diff'].value_counts(normalize=True)


On a réussi à améliorer la performance de notre prédiction, en effet elle était de 37% dans notre premier résultat et est dorénavent de 50%.
<br>
En cumul à une classe d'écart on obtient 74% de précision avec le modèle.
De plus en cumulé à deux classes d'écart on obtient 89% de précision. C'est près de 18 points de pourcentage de plus que dans le modèle précédent.

<a id="7"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Sommaire</a>
## Matrice de confusion

In [ ]:
# matrice de confusion
confusion_matrix=confusion_matrix(predict2.Response, predict2.Predicted)

In [ ]:
print(confusion_matrix)

In [ ]:
# affichage du rapport de la matrice de confusion
print(classification_report(predict2.Response, predict2.Predicted))

In [ ]:
y_actu=predict2.Response
y_pred=predict2.Predicted

In [ ]:
y_actu=pd.Series(predict2.Response, name='Réelle')
y_pred=pd.Series(predict2.Predicted, name='Prédite')
df_confusion=pd.crosstab(y_actu, y_pred)

In [ ]:
df_confusion

In [ ]:
def plot_confusion_matrix(df_confusion, title='Confusion matrix', cmap=plt.cm.Reds):
    plt.matshow(df_confusion, cmap=cmap)
    plt.title('Matrice de confusion')
    plt.colorbar()
    tick_marks = np.arange(len(df_confusion.columns))
    plt.xticks(tick_marks, df_confusion.columns, rotation=45)
    plt.yticks(tick_marks, df_confusion.index)
    #plt.tight_layout()
    plt.ylabel(df_confusion.index.name)
    plt.xlabel(df_confusion.columns.name)
    plt.show()

In [ ]:
# heatmap matrice de confusion
plot_confusion_matrix(df_confusion)

In [ ]:
# affichage des prédictions pour comparaison avec les 
plt.figure(figsize=(8, 8))
sns.heatmap(df_confusion, annot=True, fmt='d', cmap=plt.cm.Reds).set_title('Matrice des classes prédites / classes obtenues')
plt.xlabel('Classe Prédite', size=16)
plt.ylabel('Classe Réelle', size=16)
plt.title('Matrice des classes prédites / classes obtenues', size=20)


plt.show()

# Réduction avec SNE/ACP

<a id="8"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Sommaire</a>
## t-SNE

t-SNE est un algorithme de réduction de dimensions basé sur de l'apprentissage non supervisé. Il est utilisé pour de la visualisation de données ayant beaucoup de descripteurs.

Il permet de représenter les données dans un nouvel espace interprétable (2 ou 3 dimensions)
Les données proches dans l'espace original auront une probabilité élevée d'avoir une représentation proche dans le nouvel espace et à l'inverse les données éloignées ont une faible probabilité d'avoir une représentation proche dans le nouvel espace.

In [ ]:
# création d'un pipeline: évite la fuite de données
def define_preprocessor(X):
   
    # Pipeline features catégorielles
    categorical_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='constant', fill_value='missing')), # simple imputation 
            ('target_encoder', TargetEncoder()), 
            ('scaler', StandardScaler()), # standardizsation apres encodage
            ])
    
    # pipeline features numériques
    numeric_transformer = Pipeline(steps=[
            ('imputer', IterativeImputer(max_iter=10)), 
            ('scaler', StandardScaler()), # standardisation
             ])

    # pipelines features numériques et catégorielles
    preprocessor = ColumnTransformer(transformers=[
            ('cat', categorical_transformer, list(X.select_dtypes(include=['category', 'bool']).columns)),
            ('num', numeric_transformer, list(X.select_dtypes(include='number').columns)),
            ])
    
    return preprocessor

In [ ]:
# fonction préprocessing pour le SNE
def preprocessing_tSNE(dataframe, target_name='TARGET'):
    
    X = dataframe.copy()

    # suppression lignes ou il manque la valeur cible
    X = X.dropna(subset=[target_name])

    # définition variable cible
    y = X[target_name]

    # retire variable cible des feautures interprétées
    X = X.drop(columns=[target_name])

    # applique la fonction de préprocessing
    preprocessor = define_preprocessor(X)

    # applique process
    X_std = preprocessor.fit_transform(X, y)
    
    return (X_std, y)

In [ ]:
# fonction SNE, permet aussi l'affichage
def tSNE(dataframe, target_name='TARGET'):
    
    # tritement pour tSNE
    (X_std, y) = preprocessing_tSNE(dataframe, target_name)

    # Instanciation tSNE
    tsne = manifold.TSNE(n_components=2,
                         perplexity=30,
                         n_iter=300,
                         init='pca', # initialisation avec une PCA
                         random_state=0
                        )

    # Applying tSNE
    X_projected = tsne.fit_transform(X_std) 
    
    # Affichage
    plt.figure(figsize=(14,8))

    # limites graphe
    plt.xlim(X_projected[:,0].min()*1.1, X_projected[:,0].max()*1.1)
    plt.ylim(X_projected[:,1].min()*1.1, X_projected[:,1].max()*1.1)

    # définition des axes
    plt.title("t-SNE\n", fontsize=20)
    plt.xlabel("t-SNE feature 1")
    plt.ylabel("t-SNE feature 2")

    # Def nuages de points
    sc = plt.scatter(X_projected[:,0], # x
                 X_projected[:,1], #y
                 c=y,
                 cmap=plt.cm.get_cmap('RdYlGn_r'), # couleur
                 marker='.'
        )
    
    # configuration et échelle
    cbar = plt.colorbar(sc)
    cbar.ax.get_yaxis().set_ticks([])
    cbar.ax.get_yaxis().labelpad = 15
    cbar.set_label(target_name, rotation=90)

In [ ]:
data=data.drop(['Id'], axis=1)

In [ ]:
# affichage
tSNE(data, target_name='Response')

<a id="8"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Sommaire</a>
## ACP

On veut déterminer un nombre minimal de composantes à partir duquel on peut considérer que l'étude prédictive est fiable à partir d'un certain seuil (que je définis ici à 90%).
Il est important de normaliser les données pour faire une PCA (pour la conservation de la distance vectorielle).
Les données fournies ont déja été normalisée il n'est donc pas nécessaire de le faire ici. 

Il faudra néanmoins faire attention aux outliers.

In [ ]:
# Preprocessing  ACP
dataframe = data
target_name = 'Response'
(X_std, y) = preprocessing_tSNE(dataframe, target_name)

# Calcul des composantes principales

n_components=2
pca = decomposition.PCA(n_components=n_components)
pca.fit(X_std)

print("Pourcentage variance expliquée par composante:", pca.explained_variance_ratio_)
print("Pourcentage total variance expliquée:", pca.explained_variance_ratio_.sum()) #  somme cumulée 

In [ ]:
# représentation de l'ACP avec deux composantes
pca=PCA(n_components=2)
components=pca.fit_transform(X_std)
fig = px.scatter(components, x=0, y=1, color=data.Response, title='Représentation PCA 2 composantes')
fig.show()

In [ ]:
# représentation de l'ACP avec 3 composantes
pca = PCA(n_components=3)
components = pca.fit_transform(X_std)
total_var = pca.explained_variance_ratio_.sum() * 100
fig = px.scatter_3d(
    components, x=0, y=1, z=2, color=data.Response,
    title='PCA avec 3 composantes\nVariance explicative cumulée:: {}%'.format(total_var),
    labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'}
)
fig.show()

In [ ]:
# fonction explicative des valeur propres pour PCA
def display_scree_plot(X_std):
    pca = decomposition.PCA()
    pca.fit(X_std)
    scree = pca.explained_variance_ratio_*100
    
    plt.bar(np.arange(len(scree))+1, scree)
    plt.plot(np.arange(len(scree))+1, scree.cumsum(),c="red",marker='o')
    plt.xlabel("Quantité de composantes")
    plt.ylabel("pourcentage de variance cumulée")
    plt.title("Etude du seuil de variance en fonction du nombre de composantes pour la PCA", fontsize=15)
    plt.grid(linestyle='dotted')
    plt.show(block=False)

In [ ]:
# affichage valeurs propres PCA
display_scree_plot(X_std)

In [ ]:
# fonction donne les composantes principales de l'ACP, jusqu'au seuil de variance
def PCA_features_reduction(X_std, var_threshold=0.9): 
    # PCA
    pca = decomposition.PCA()
    pca.fit(X_std)
    
    # ratio de variance expliqué pour chaque composante principale
    scree = pca.explained_variance_ratio_
    # rend le nombre de composants principaux pour atteindre les seuils de variance
    mask = scree.cumsum() > var_threshold
    nb_selected_features = len(scree[~mask]) + 1
    print("Nombre de features selectionnées:", nb_selected_features)
    
    # Calcul du ratio
    explained_variance_sum = scree.cumsum()[nb_selected_features-1]
    print("Valeur cumulée de variance expliquée:  {:.2f}%".format(explained_variance_sum*100))
    
    # projection sur les 1ers composant
    X_projected = pca.transform(X_std)[:,:nb_selected_features]
    
    return X_projected

In [ ]:
X_projected = PCA_features_reduction(X_std, var_threshold=0.9)
X_projected.shape

Le seuil de 90% de variance expliquée est atteint avec 78 composantes principales.
On avait initialement près de 120 variables.

## Feature importance pour la PCA

In [ ]:
pca.components_

In [ ]:
X_std

In [ ]:
test_df=data.copy()
# crée objet standardscaler
sc=StandardScaler()
#stardisation données
X_test_std=sc.fit_transform(test_df)
# applique PCA
pca=PCA()
X_test_pca=pca.fit(X_test_std)

In [ ]:
# reduction dimensionnelle à 78 composantes
num_components = 78
pca = PCA(num_components)  
X_test_pca = pca.fit_transform(X_test_std)
# recherche des features le plus influentes
n_pcs= pca.n_components_ # get number of component
# get the index of the most important feature on EACH component
most_important = [np.abs(pca.components_[i]).argmax() for i in range(n_pcs)]
initial_feature_names = test_df.columns
# get the most important feature names
most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]
most_important_names

In [ ]:
# pour 3 composantes
model = PCA(n_components=3).fit(X_std)
X_pc = model.transform(X_std)
n_pcs = model.components_.shape[0]
most_important = [np.abs(model.components_[i]).argmax() for i in range(n_pcs)]
dic = {'PC{}'.format(i): most_important_names[i] for i in range(n_pcs)}
dic

In [ ]:
# matrice de covariance
cov_matrix=np.cov(X_std.T)
print('Dimensions de la matrice:', cov_matrix.shape)

In [ ]:
# valeur propres de la matrice et vecteurs propres
eigenvalues, eigenvectors=np.linalg.eig(cov_matrix)

In [ ]:
# classement des valeurs propres

#on crée une paire valeur propre/vecteur propre
eig_pairs=[(eigenvalues[index], eigenvectors[:, index]) for index in range(len(eigenvalues))]

# classement des paire décroissant
eig_pairs.sort()
eig_pairs.reverse()

#extraction
eigvalues_sorted = [eig_pairs[index][0] for index in range(len(eigenvalues))]
eigvectors_sorted = [eig_pairs[index][1] for index in range(len(eigenvalues))]

print('Valeurs propres décroissantes:\n {}' .format(eigvalues_sorted))

In [ ]:
# variance expliquée de chaque composante (numériquement ordonnée)
tot = sum(eigenvalues)
var_explained = [(i / tot) for i in sorted(eigenvalues, reverse=True)]
cum_var_exp = np.cumsum(var_explained)
print(var_explained)

In [ ]:
# vérification variance cumulée
print('Variance cumulée: {}'.format(cum_var_exp))

In [ ]:
# on réduit a 8 dimension pour l'affichage du pairplot

P_reduce = np.array(eigvectors_sorted[0:8])
X_std_8D = np.dot(X_std,P_reduce.T)
reduced_pca = pd.DataFrame(X_std_8D)
reduced_pca

In [ ]:
# affichage pairplot
sns.pairplot(reduced_pca, diag_kind='kde') 
plt.show()

In [ ]:
# importance des variables avec un arbre de regression aléatoire

data_prediction=data.copy()
data_prediction=data_prediction.drop(['Response'], axis=1)



from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(random_state=42, max_depth=10)
model.fit(data_prediction,y)
features = data.columns
importances = model.feature_importances_
indices = np.argsort(importances)[-9:]  # top 10 des variables

plt.title('Importance des variables donné par le Random Forest', size=20)
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Importance relative')
plt.show()

In [ ]:
# indice des 34 variables les moins influentes selon le Random Forest 
var_to_drop=np.argsort(importances)[78:]
var_to_drop

In [ ]:
features_to_drop=[
    'Medical_History_35',
    'Medical_History_38',

    'Medical_Keyword_20',
    'Medical_Keyword_44',
    'Medical_Keyword_8',
    'Medical_Keyword_13',
    'Medical_Keyword_26',
    'Medical_Keyword_39',
    'Medical_Keyword_6',
    'Medical_Keyword_5',
    'Medical_Keyword_18',
    'Medical_Keyword_35',
    'Medical_Keyword_21',
    'Medical_Keyword_19',
    'Medical_Keyword_36',
    'Medical_Keyword_12',
    'Medical_Keyword_45',
    'Medical_Keyword_27',
    'Medical_Keyword_17',
    'Medical_Keyword_14',
    'Medical_Keyword_16',
    'Medical_Keyword_2',
    'Medical_Keyword_29',
    'Medical_Keyword_32',
    'Medical_Keyword_30',
    'Medical_Keyword_31',
    'Medical_Keyword_11',
    'Medical_Keyword_34',
    'Medical_Keyword_7',
    'Medical_Keyword_24',

    'Product_Info_5',
    'Product_Info_1',
    'Product_Info_7'

]


In [ ]:
# suppression des variables 
for feature in features_to_drop:
    data_prediction=data_prediction.drop(feature, axis=1)

In [ ]:
data_prediction.shape
data_prediction

In [ ]:
features_prediction=list(data_prediction.columns)
X_prediction=data[features_prediction]
print(X_prediction.shape)
print(y.shape)

In [ ]:
Bayes3 = MultinomialNB()
Bayes3.fit(X_prediction,y)

In [ ]:
# affichage comparatif
predict3=pd.DataFrame(data=Bayes3.predict(X_prediction))
predict3.rename(columns={0:'Predicted'}, inplace=True)
predict3['Response']=y
predict3

In [ ]:
# nombre des valeurs pour la variable prédite
predict3.Predicted.value_counts()

In [ ]:
# valeur des prédictions
sns.countplot(data=predict3, x='Predicted').set_title("Prédiction pour chaque catégorie pour le 3ème modèle")
plt.grid(linestyle='dotted')
plt.show()

In [ ]:
# comparaison du modèle avec la valeurs réelles
fig, axes = plt.subplots(1,2,figsize=(16,6))
fig.suptitle('Comparaison classes prédites vs classes réelles 3eme modele')
sns.countplot(ax=axes[0], data=predict3, x='Predicted')
sns.countplot(ax=axes[1], data=predict3, x='Response')
plt.show()

In [ ]:
# différence entre la variable prédite et Response
predict3['diff']=abs(predict3.Predicted-predict3.Response)
predict3.head(5)

In [ ]:
# différence de classe entre prédiction et Response
predict3['diff'].value_counts()

In [ ]:
# quantité de données différence classes prédiction/réelles
sns.countplot(data=predict3, x='diff').set_title("classes d'écart entre valeurs prédites et valeurs réelles 3eme modele")
plt.grid(linestyle='dotted')
plt.show()

In [ ]:
# pourcentage de classes écart entre prédiction et 'Response'
predict3['diff'].value_counts(normalize=True)

Interprétation des résultats: 
- 36% de prédiction exacte
- 17% de prédiction à 2 classes d'écart
- 17% de prédiction à 1 classe d'écart
- on n'a pas plus de 7% prédiction à plus de 2 classes d'écart

## Métrique de comparaison

In [ ]:
from ml_metrics import quadratic_weighted_kappa
def eval_wrapper(yhat, y):  
    y = np.array(y)
    y = y.astype(int)
    yhat = np.array(yhat)
    yhat = np.clip(np.round(yhat), np.min(y), np.max(y)).astype(int)   
    return quadratic_weighted_kappa(yhat, y)

## Autres modèles prédictifs

In [ ]:
pip install xgboost

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

from sklearn import ensemble
from sklearn import metrics
from sklearn.metrics import classification_report, recall_score, accuracy_score, precision_score
from sklearn.model_selection import train_test_split

In [ ]:
data_prediction

In [ ]:


X_full = pd.read_csv('../input/prudential-life-insurance-assessment/train.csv.zip', index_col='Id')
X_test_full = pd.read_csv('../input/prudential-life-insurance-assessment/test.csv.zip', index_col='Id')


In [ ]:
y = X_full.Response
X = X_full.drop(labels=['Response'],axis=1)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size=.30,random_state=42)

In [ ]:
# suppression variable catégorielle
X_dropped_train=X_train.drop(axis=1,labels=["Product_Info_2"]).copy()
X_dropped_valid=X_valid.drop(axis=1,labels=["Product_Info_2"]).copy()

In [ ]:
#suppression des colonnes avec des null
X_dropped_train.dropna(axis=1,inplace=True)
X_dropped_valid.dropna(axis=1,inplace=True)

In [ ]:
print("Dimension X_train {}".format(X_dropped_train.shape))
print("Dimension X_test {}".format(X_dropped_valid.shape))

print("Dimension y_train {}".format(y_train.shape))
print("Dimension y_valid {}".format(y_valid.shape))

In [ ]:
seed=42

# liste des modèles
dt=DecisionTreeClassifier(random_state=seed)
rf=RandomForestClassifier(random_state=seed)
lr=LogisticRegression(random_state=seed)
adb=ensemble.AdaBoostClassifier()
bgc=ensemble.BaggingClassifier()
gbc=ensemble.GradientBoostingClassifier()
xgb=XGBClassifier(random_state=seed)
svc=SVC(random_state=seed)

## Random Forest

In [ ]:

X_train, X_valid, y_train, y_valid = train_test_split(data_prediction,y,test_size=.30,random_state=42)

X_dropped_train=X_train.dropna(axis=1,inplace=True)
X_dropped_valid=X_valid.dropna(axis=1,inplace=True)

In [ ]:
data_prediction

In [ ]:
predict2

In [ ]:
eval_wrapper(predict2.Response, predict2.Predicted)

In [ ]:
eval_wrapper(predict_df.Response, predict_df.Predicted)

In [ ]:
eval_wrapper(predict3.Response, predict3.Predicted)

In [ ]:
predict_df

In [ ]:
final_df

In [ ]:
import xgboost as xgb
xgb_num_rounds = 720
num_classes = 8
missing_indicator = -1000

In [ ]:
train = pd.read_csv('../input/prudential-life-insurance-assessment/train.csv.zip')
test = pd.read_csv('../input/prudential-life-insurance-assessment/test.csv.zip')

In [ ]:
def get_params():
    
    params = {}
    params["objective"] = "reg:squarederror"     
    params["eta"] = 0.05
    params["min_child_weight"] = 360
    params["subsample"] = 0.85
    params["colsample_bytree"] = 0.3
    params["silent"] = 1
    params["max_depth"] = 7
    plst = list(params.items())

    return plst



In [ ]:
plst=get_params()
plst

In [ ]:
final_df

In [ ]:
all_data = train.append(test)
all_data=all_data.drop(['Product_Info_2'], axis=1)
all_data

In [ ]:
all_data = all_data.drop(cols_with_missing, axis=1)

In [ ]:
all_data.fillna(missing_indicator, inplace=True)
all_data

In [ ]:
all_data['Response'] = all_data['Response'].astype(int)


In [ ]:

train = all_data[all_data['Response']>0].copy()
test = all_data[all_data['Response']<1].copy()

In [ ]:
columns_to_drop=['Response']
xgtrain = xgb.DMatrix(train.drop(columns_to_drop, axis=1), train['Response'].values, 
                        missing=missing_indicator)
xgtest = xgb.DMatrix(test.drop(columns_to_drop, axis=1), label=test['Response'].values, 
                        missing=missing_indicator) 

In [ ]:
model = xgb.train(plst, xgtrain, xgb_num_rounds) 


In [ ]:
train_preds = model.predict(xgtrain, ntree_limit=model.best_iteration)
print('Train score is:', eval_wrapper(train_preds, train['Response'])) 
test_preds = model.predict(xgtest, ntree_limit=model.best_iteration)


In [ ]:
from scipy.optimize import fmin_powell

def apply_offsets(data, offsets):
    for j in range(num_classes):
        data[1, data[0].astype(int)==j] = data[0, data[0].astype(int)==j] + offsets[j]
    return data

def score_offset(data, bin_offset, sv, scorer=eval_wrapper):
    # data has the format of pred=0, offset_pred=1, labels=2 in the first dim
    data[1, data[0].astype(int)==sv] = data[0, data[0].astype(int)==sv] + bin_offset
    score = scorer(data[1], data[2])
    return score


offsets = np.array([0.1, -1, -2, -1, -0.8, 0.02, 0.8, 1])
offset_preds = np.vstack((train_preds, train_preds, train['Response'].values))
offset_preds = apply_offsets(offset_preds, offsets)
opt_order = [6,4,5,3]
for j in opt_order:
    train_offset = lambda x: -score_offset(offset_preds, x, j) * 100
    offsets[j] = fmin_powell(train_offset, offsets[j], disp=False)


In [ ]:
print('Offset Train score is:', eval_wrapper(offset_preds[1], train['Response'])) 


In [ ]:
data = np.vstack((test_preds, test_preds, test['Response'].values))
data = apply_offsets(data, offsets)

final_test_preds = np.round(np.clip(data[1], 1, 8)).astype(int)

preds_out = pd.DataFrame({"Id": test['Id'].values, "Predicted": final_test_preds})
preds_out = preds_out.set_index('Id')

In [ ]:
preds_out